In [10]:
import gpxpy
import gpxpy.gpx

import folium
import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

with open('Day_nine_arcata_.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)



route_info = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            route_info.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation
            })


route_df = pd.DataFrame(route_info)
route_df.head()


route_map = folium.Map(
    location=[41.100848,-124.159372],
    zoom_start=10,
    tiles='CartoDBPositron',
    width=1024,
    height=600
)
coordinates = [tuple(x) for x in route_df[['latitude', 'longitude']].to_numpy()]
folium.PolyLine(coordinates, weight=5).add_to(route_map)


display(route_map)